In [1]:
import pandas as pd
import numpy as np
import datetime
import io
import requests
import streamlit as st
from st_aggrid import GridOptionsBuilder, AgGrid, JsCode
from st_aggrid.shared import ColumnsAutoSizeMode
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)
simplefilter(action='ignore', category=Warning)

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.expand_frame_repr', False)

In [2]:
import streamlit_authenticator as stauth
hashed_passwords = stauth.Hasher(['12345', '456']).generate()
hashed_passwords

['$2b$12$PMk9DyCzggK7mU3GVlIq8OnCRTb227tVyQOU1VU5v2FhJAI8B45vq',
 '$2b$12$22LKJU6uvK7BQ8rQQglULu1HEN0iUnBV8gn0vsIEqn0QNsq5u3WKu']

In [3]:
import funcoes as f

produtos = f.produtos()
produtos
# filter STATUS_FINAL == ATIVO
produtos = produtos[produtos['STATUS_FINAL'] == 'ATIVO']
produtos





,Cod. Modelo,Cod. Gr.,Desc. Gr.,Cod. Subgr.,Desc. Subgr.,Cod. Prod.,Desc. Produto,Markup Aplic,Pr. Custo,Pr Venda unit,Faixa de Preco,Gde. Gr.,Metal,Cor Padrao,Desc Cor,Sub Nivel,Tab. Pr.,Foto,Encomendavel,Status,Estoque,STATUS_FINAL,Data Inclusao,Peso,Classificacao,Linha
19,15 Anos,AN,ANEL,DN,DIAMANTE NEGRO,AN02482,"ANEL OBN, STAR DIAMOND, DN ±0,82CT",4.23,483.02,11000.0,06. De 10.000 a 20.000,PA,OURO,RN,RODIO NEGRO,NaN,A,https://jvphotos.com.br/cms/wp-content/uploads...,Sim,Ativo,0,ATIVO,2015-05-26,8.00,COM,ANEL CRAV
20,15 Anos,AN,ANEL,DI,DIAMANTE INCOLOR,AN02483,"ANEL OB, STAR DIAMOND, DIA ±0,74CT",4.27,804.97,18500.0,06. De 10.000 a 20.000,PA,OURO,OB,OURO BRANCO,NaN,A,https://jvphotos.com.br/cms/wp-content/uploads...,Sim,Ativo,0,ATIVO,2015-05-26,8.00,COM,ANEL CRAV
21,15 Anos,AN,ANEL,DI,DIAMANTE INCOLOR,AN02949,"ANEL OBN, STAR DIAMOND, DLLB ±0,74CT",4.26,645.57,14800.0,06. De 10.000 a 20.000,PA,OURO,RN,RODIO NEGRO,NaN,A,https://jvphotos.com.br/cms/wp-content/uploads...,Sim,Ativo,4,ATIVO,2017-04-05,8.00,COM,ANEL CRAV
31,15 Anos,BR,BRINCO,DI,DIAMANTE INCOLOR,BR04014T,"BR OB, STAR DIAMOND P , D 0,49CT",4.42,635.17,15200.0,06. De 10.000 a 20.000,PA,OURO,OB,OURO BRANCO,BR COMETA,A,https://jvphotos.com.br/cms/wp-content/uploads...,Sim,Ativo,2,ATIVO,2014-10-14,6.00,COM,BRINCO MAN
37,15 Anos,BR,BRINCO,DN,DIAMANTE NEGRO,BR04027T,"BR OBN, STAR DIAMOND P , DN 0,52CT",4.43,412.77,9900.0,07. De 5.000 a 10.000,PA,OURO,RN,RODIO NEGRO,NaN,A,https://jvphotos.com.br/cms/wp-content/uploads...,Sim,Ativo,0,ATIVO,2014-11-03,6.00,COM,BRINCO MAN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16675,WE LOVE SA,PIE,PIERCING,DI,DIAMANTE INCOLOR,PI01457,"PIER OB,PIER SAPPHIRE DIAMANTE,D ±0,30CT",4.66,273.53,6800.0,07. De 5.000 a 10.000,PA,OURO,OB,OURO BRANCO,PIERCING,A,https://jvphotos.com.br/cms/wp-content/uploads...,Sim,Ativo,0,ATIVO,2022-04-07,1.44,COM,BRINCO CRA
16679,WHITECHAIN,AN,ANEL,SP,SEM PEDRA,AN03175,"ANEL PR, CHAIN, BANHO RB",22.22,9.22,1100.0,03. Ate 2.000,PA,PRATA,OB,OURO BRANCO,ALIANÇA,A,https://jvphotos.com.br/cms/wp-content/uploads...,Nao,Inativo,9,ATIVO,2019-01-07,2.50,COM,ANEL S/P
16683,WHITECHAIN,BR,BRINCO,SP,SEM PEDRA,BR05598T,"BRINCO PR, CHAIN, 11CM, 1 PÉ, BANHO RB",8.30,28.03,1250.0,03. Ate 2.000,PA,PRATA,OB,OURO BRANCO,NaN,A,https://jvphotos.com.br/cms/wp-content/uploads...,Nao,Inativo,7,ATIVO,2019-01-23,7.20,COM,BRINCO S/P
16684,WHITECHAIN,BR,BRINCO,SP,SEM PEDRA,BR05599T,"BRINCO PR, CHAIN, 5CM, 1 PÉ, BANHO RB",11.18,16.33,980.0,03. Ate 2.000,PA,PRATA,OB,OURO BRANCO,NaN,A,https://jvphotos.com.br/cms/wp-content/uploads...,Nao,Inativo,1,ATIVO,2019-01-23,3.30,COM,BRINCO S/P


In [5]:
url = "https://jvphotos.com.br/cms/wp-content/uploads/datasets/ultima_fase_producao_gzip.csv"
headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.14; rv:66.0) Gecko/20100101 Firefox/66.0"
}
response = requests.get(url, headers=headers)
df = pd.read_csv(
    io.BytesIO(response.content), sep=";", compression="gzip", low_memory=True
)
df = df[~df['Fase'].str.contains('ENCERRA')]
df = df[(df['Peso_Fase'] != 10)]
df = df[(df["Fase"] != "ORDEM DE PRODUÇÃO")]
df['Abertura_Pedido'] = pd.to_datetime(df['Abertura_Pedido'], format='%Y-%m-%d')
df['Processamento_op'] = pd.to_datetime(df['Processamento_op'], format='%Y-%m-%d')
df['Data'] = pd.to_datetime(df['Data'], format='%Y-%m-%d')
df['Ano_Processamento'] = df['Processamento_op'].dt.year
df = df[df['Ano_Processamento'] > 2021]
df = df[df['Cod_Prod'] == 'BR06544T']

colunasRemover = ['OperacaoProd', 'Data', 'Abertura_Pedido', 'Processamento_op', 'Dias em aberto',  'num OP', 'Grupo Destino', 'Conta Destino', 'Tab. Pr.', 'Metal', 'Gde. Gr.', 'Peso']
df = df.drop(colunasRemover, axis=1)

# df = (
#     df.rename(columns={"Qtd": "Producao Iniciada"})
#     .groupby(["Cod_Prod"], as_index=False)["Producao Iniciada"]
#     .sum()
# )

# df = df[df["Producao Iniciada"] > 0].reset_index(drop=True)
df = df.sort_values(by=["OP"])
df

,Desc_Operacao,Fase,OP,Num_Pedido,Qtd,Cod_Prod,Peso_Fase_2,Peso_Fase,Ano_Processamento
7983,PED PRODUCAO TAB B,MOVIMENTACAO ESTOQUE,512700003,1017,1,BR06544T,1,2,2022
6849,PEDIDO PROD GERADOR,MUDANCA DE FASE,518630018,384,1,BR06544T,2,2,2023
7139,PEDIDO PROD GERADOR,MUDANCA DE FASE,519990003,391,1,BR06544T,2,2,2023
7194,PEDIDO PROD GERADOR,MOVIMENTACAO ESTOQUE,519990019,391,1,BR06544T,1,2,2023


In [6]:
url = "https://jvphotos.com.br/cms/wp-content/uploads/datasets/ultima_fase_producao_gzip.csv"
headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.14; rv:66.0) Gecko/20100101 Firefox/66.0"
}
response = requests.get(url, headers=headers)
df = pd.read_csv(io.BytesIO(response.content), sep=";", compression="gzip", low_memory=True)

df = df[~df['Fase'].str.contains('ENCERRA')]

# df = df[(df["Fase"] == "ORDEM DE PRODUÇÃO")]
# df = (
#     df.rename(columns={"Qtd": "Producao Nao Iniciada"})
#     .groupby(["Cod_Prod"], as_index=False)["Producao Nao Iniciada"]
#     .sum()
# )
# df = df[df["Producao Nao Iniciada"] > 0].reset_index(drop=True)

# df = peso fase = 10

df = df[(df['Peso_Fase'] != 10)]
df = df[df['Fase'] == 'ORDEM DE PRODUÇÃO']
df['Abertura_Pedido'] = pd.to_datetime(df['Abertura_Pedido'], format='%Y-%m-%d')
df['Processamento_op'] = pd.to_datetime(df['Processamento_op'], format='%Y-%m-%d')
df['Data'] = pd.to_datetime(df['Data'], format='%Y-%m-%d')

df['Ano_Processamento'] = df['Processamento_op'].dt.year

df = df[df['Ano_Processamento'] > 2021]
df = df[df['Cod_Prod'] == 'BR06544T']

colunasRemover = ['OperacaoProd', 'Data', 'Abertura_Pedido', 'Processamento_op', 'Dias em aberto',  'num OP', 'Grupo Destino', 'Conta Destino', 'Tab. Pr.', 'Metal', 'Gde. Gr.', 'Peso']
df = df.drop(colunasRemover, axis=1)

df = df.sort_values(by=["OP"])
df

,Desc_Operacao,Fase,OP,Num_Pedido,Qtd,Cod_Prod,Peso_Fase_2,Peso_Fase,Ano_Processamento
8326,PEDIDO PROD GERADOR,ORDEM DE PRODUÇÃO,519990018,391,1,BR06544T,0,0,2023


In [ ]:
ano_mes_3_meses = df[df['Ano_Mes'] < ano_mes_hoje].sort_values(by='Ano_Mes', ascending=False)['Ano_Mes'].unique()[2]
ano_mes_3_meses

In [ ]:
import xlsxwriter

# Note the file extension should be .xlsm.
workbook = xlsxwriter.Workbook('macros.xlsm')
# worksheet = workbook.add_worksheet()
workbook.vba_project = './vbaProject.bin'
# worksheet.set_column('A:A', 30)

# # Add the VBA project binary.
workbook.add_vba_project('./Fotos.bin')

# # Show text for the end user.
# worksheet.write('A3', 'Press the button to say hello.')

# # Add a button tied to a macro in the VBA project.
# worksheet.insert_button('B3', {'macro':   'say_hello',
#                                'caption': 'Press Me',
#                                'width':   80,
#                                'height':  30})

# workbook.close()

In [ ]:
url = "https://jvphotos.com.br/cms/wp-content/uploads/datasets/clientes_rfv_gzip.csv"
headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.14; rv:66.0) Gecko/20100101 Firefox/66.0"
}
response = requests.get(url, headers=headers)
df = pd.read_csv(
    io.BytesIO(response.content), sep=";", compression="gzip", low_memory=False
)

df.drop(columns=['Proxima Compra', 'Recencia', 'Frequencia', 'Valor'], inplace=True)
df.fillna(0, inplace=True)

DateColumns = ['Primeira compra', 'Ultima compra']
for col in DateColumns:
    df[col] = pd.to_datetime(df[col])

df['Ano_ultima_compra'] = df['Ultima compra'].dt.year
df['Anos_Marca'] = (datetime.datetime.now() - df['Primeira compra']).dt.days / 365

intColumns = ['Total liq.', 'Qtd tickets', 'Ticket medio', 'Dias ultima compra', 'Media entre compras', 'Anos_Marca']
for col in intColumns:
    df[col] = df[col].astype(int)


In [ ]:
url = "https://jvphotos.com.br/cms/wp-content/uploads/datasets/eletroformacao_gzip.csv"
headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.14; rv:66.0) Gecko/20100101 Firefox/66.0"}
response = requests.get(url, headers=headers)
df = pd.read_csv(io.BytesIO(response.content), sep=';', compression='gzip', low_memory=False)
df

In [ ]:
produtos_eletro = produtos.join(df.set_index('cpros'), on='Cod. Prod.', how='inner')
produtos_eletro.shape

In [ ]:
url = "https://jvphotos.com.br/cms/wp-content/uploads/datasets/estoque_venda_gzip.csv"
headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.14; rv:66.0) Gecko/20100101 Firefox/66.0"}
response = requests.get(url, headers=headers)
df = pd.read_csv(io.BytesIO(response.content), sep=";", compression="gzip", low_memory=True)

df = df.groupby('Cod. Prod.').sum().reset_index()
df.rename(columns={'Cod. Prod.': 'Cod_Prod', 'Qtd': 'Estoque'}, inplace=True)
df

In [ ]:
url = "https://jvphotos.com.br/cms/wp-content/uploads/datasets/ultima_fase_producao_gzip.csv"
headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.14; rv:66.0) Gecko/20100101 Firefox/66.0"}

with requests.get(url, headers=headers, stream=True) as r:
    r.raise_for_status()
    with io.BytesIO() as b:
        for chunk in r.iter_content(chunk_size=8192):
            if chunk:
                b.write(chunk)
        b.seek(0)
        df = pd.read_csv(b, sep=';', compression='gzip', low_memory=False)

df = df[(df['Peso_Fase'] != 10) & (df['Fase'] != 'ORDEM DE PRODUÇÃO')]
df = df.rename(columns={'Qtd': 'Producao Iniciada'}).groupby(['Cod_Prod'], as_index=False)['Producao Iniciada'].sum()
df = df[df['Producao Iniciada'] > 0].reset_index(drop=True)
df.head()

In [ ]:
# producao
url = "https://jvphotos.com.br/cms/wp-content/uploads/datasets/ultima_fase_producao_gzip.csv"
headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.14; rv:66.0) Gecko/20100101 Firefox/66.0"}
response = requests.get(url, headers=headers)
df = pd.read_csv(io.BytesIO(response.content), sep=';', compression='gzip', low_memory=False)
df = df[(df['Peso_Fase'] != 10) & (df['Fase'] == 'ORDEM DE PRODUÇÃO')]
df = df.rename(columns={'Qtd': 'Producao Nao Iniciada'}).groupby(['Cod_Prod'], as_index=False)['Producao Nao Iniciada'].sum()
df = df[df['Producao Nao Iniciada'] > 0].reset_index(drop=True)
df.head()

In [ ]:
colunasInt = ['Qtd', 'Total Liq.', 'Desconto', 'Custo', 'Total Brt']
colunasStr = ['Cod. Barras', 'No.Oper']
df[colunasInt] = df[colunasInt].astype('int')
df[colunasStr] = df[colunasStr].astype('str')

In [ ]:
df['Data'] = pd.to_datetime(df['Data']).dt.date


In [ ]:
df.dtypes

In [ ]:
url = "https://jvphotos.com.br/cms/wp-content/uploads/datasets/METAS.xlsx"
headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.14; rv:66.0) Gecko/20100101 Firefox/66.0"}
response = requests.get(url, headers=headers)
df = pd.read_excel(io.BytesIO(response.content), sheet_name='Metas')

In [ ]:
# rename columns
colunas = ['Loja', 'Data', 'Cod_Vend', 'Consultor', 'Meta']
df.rename(columns=dict(zip(df.columns, colunas)), inplace=True)
df.drop(columns=['METAL PROPORCIONAL'], inplace=True)
df['Meta'] = df['Meta'].astype('int')

In [ ]:
# df['Meta'] = df['Meta'].astype(int)
df['Data'] = pd.to_datetime(df['Data']).dt.date
df

In [ ]:
products = {
    "No.": [1, 2, 3],
    "Name": [' Chair', ' Cabinet', ' Table'],
    "Price": [4, 12, 10],
    "Stocks": [100, 50, 60],
    "Details": [
        "https://i.imgur.com/fH2LHvo.png",
        "https://i.imgur.com/bvHZX5j.png",
        "https://i.imgur.com/D7xDwT9.png"
    ]
}

df = pd.DataFrame(products)

ob = GridOptionsBuilder.from_dataframe(df)

# 1. Add an image next to the value under the Name column.
#    Also add a link on the label.
image_cr = JsCode("""
    function(params) {
        var element = document.createElement("span");

        var imageElement = document.createElement("img");
        var anchorElement = document.createElement("a");

        // Update the image element. Use the value from Details column.
        imageElement.src = params.data.Details;
        imageElement.width="80";
        imageElement.height="80";
        element.appendChild(imageElement);

        // Add a link to the Name value. The link is from the Details column.
        anchorElement.href = params.data.Details;
        anchorElement.target = "_blank";
        anchorElement.innerHTML = params.value;
        element.appendChild(anchorElement);

        return element;
    }""")
ob.configure_column('Name', cellRenderer=image_cr)

# 2. Configure the column Details. Add a link to inner value.
image_url = JsCode("""
    function(params) {
        return `<a href=${params.value} target="_blank">${params.value}</a>`
    }""")
ob.configure_column("Details", cellRenderer=image_url)

# 2.1. Style the cell, if stocks is below 60, increase font-size
#      and color it red - as a warning to supply department.
low_supply = JsCode("""
    function(params) {
        if (params.value < 60) {
            return {
                'color': 'red',
                'font-size': '20px'
            };
        }
    }""")
ob.configure_column("Stocks", cellStyle=low_supply)

# 3. Update selection.
ob.configure_selection(selection_mode="multiple", use_checkbox=True)

# 4. Update row height, to make the image look clearer.
ob.configure_grid_options(rowHeight=100)

# 5. Hide the Details column. The product name has a link already.
ob.configure_column("Details", hide=True)

# 6. Build the options.
grid_options = ob.build()

st.markdown('#### Streamlit-AgGrid')

# 7. Add custom css to center the values, as we adjusted the row height in step 4.
grid_return = AgGrid(
    df,
    grid_options,
    allow_unsafe_jscode=True,
    enable_enterprise_modules=False,
    columns_auto_size_mode=ColumnsAutoSizeMode.FIT_ALL_COLUMNS_TO_VIEW,
    key='products',
    custom_css={'.ag-row .ag-cell': {'display': 'flex',
                                     'justify-content': 'center',
                                     'align-items': 'center'},
                '.ag-header-cell-label': {'justify-content': 'center'}}
) 

selected_rows = grid_return["selected_rows"]

if len(selected_rows):
    st.markdown('#### Selected')
    dfs = pd.DataFrame(selected_rows)

    dfsnet = dfs.drop(columns=['_selectedRowNodeInfo', 'Details'])
    AgGrid(
        dfsnet,
        enable_enterprise_modules=False,
        columns_auto_size_mode=ColumnsAutoSizeMode.FIT_ALL_COLUMNS_TO_VIEW,
        reload_data=True,
        key='product_selected'
    )